In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
data_path = './outlier/train102.csv'

In [3]:
def read_data(data_path):
    df = pd.read_csv(data_path)
    value = np.array(df['Value'])
    label = np.array(df['Label'])
    return value, label

In [4]:
value, label = read_data(data_path)
print(value.size)
print(label.size)

127130
127130


In [5]:
WINDOW_SIZE = 128
EPOCH = 20
BATCH_SIZE = 256
LEARNING_RATE = 1e-3

In [6]:
def normalize_data(value):
    mean = np.mean(value)
    std = np.std(value)
    print(mean)
    print(std)
    value = [(x - mean) / std for x in value]
    return value

value = normalize_data(value)
#print(value)

3038.0579182726347
565.2425692392306


In [7]:
# value = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13])
# label = np.array([0,1,0,1,0,1,0,1,0,1,0,1,0])
# WINDOW_SIZE = 3
# BATCH_SIZE = 2

def create_window(value, label):
    windows = np.array([value[:WINDOW_SIZE]])
    for i in range(len(value) - WINDOW_SIZE):
        windows = np.concatenate((windows, np.array([value[i + 1:i + 1 + WINDOW_SIZE]])))
    label = label[WINDOW_SIZE - 1:]
    label = label[:,np.newaxis]
    return windows, label
windows, label = create_window(value, label)

print(len(windows))
# print(windows)
# print(label)

127003


In [11]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils import clip_grad_norm_

from tqdm import tqdm
from sklearn.utils import shuffle

In [9]:
sep1 = int(0.6 * len(windows))
sep2 = int(0.8 * len(windows))
# print(sep1)
# print(sep2)

windows, label = shuffle(windows, label)

windows = torch.from_numpy(windows).float()
label = torch.from_numpy(label).float()

# print(windows)
# print(label)

train_set = TensorDataset(windows[:sep1], label[:sep1])
train_data = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
print(len(train_data))

valid_set = TensorDataset(windows[sep1:sep2], label[sep1:sep2])
valid_data = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
print(len(valid_data))

# test set 不需要打乱，搞成batch？
# test_set = TensorDataset(windows[sep2:], label[sep2:])
# test_data = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
# print(len(test_data))

# for i, batch in enumerate(train_data):
#     print(i)
#     # batch[0]为数据,batch[1]为标签
#     print(batch[0])
#     print(batch[1])
    

297
99


In [13]:
from AE import AE
net = AE()
# print(net)
if torch.cuda.is_available():
    net = net.cuda()

optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE, weight_decay=1e-3) # 添加正则项，替代dropout

# optimizer.param_groups[0]
# 是其中一个参数组，包括['lr']和['weight_decay']
# 可以直接赋值修改参数
# 防止有多个参数组，故使用循环
# def set_learning_rate(optimizer, lr):
#     for param_group in optimizer.param_groups:
#         param_group['lr'] = lr

# 每10轮衰减为原来的75%
def set_learning_rate(optimizer):
    for param_group in optimizer.param_groups:
        param_group['lr'] *= 0.75

train_losses = []
valid_losses = []

prev_time = datetime.now()

for e in tqdm(range(EPOCH)):
    if e != 0 and e % 10 == 0:
        set_learning_rate(optimizer)
        
    train_loss = 0.0
    for i, batch in enumerate(train_data):
        window = batch[0].view(BATCH_SIZE, -1)
        recon_window = net(window)
        loss = net.loss_function(recon_window, window) / BATCH_SIZE
        
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(net.parameters(), max_norm=10.)
        optimizer.step()
        # use float() to reduce menmery 'autograd
        train_loss += float(loss)
        
    train_loss /= len(train_data)
    train_losses.append(train_loss)
    
    valid_loss = 0
    
    net.eval()
    
    for i, batch in enumerate(valid_data):
        window = batch[0].view(BATCH_SIZE, -1)
        recon_window = net(window)
        loss = net.loss_function(recon_window, window) / BATCH_SIZE
        valid_loss = float(loss)
        break
        
    valid_losses.append(valid_loss)
        
    if (e + 1) % 5 == 0:
        cur_time = datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        
        prev_time = cur_time
        #print('Epoch: {}, Train Loss: {:.4f} '.format(e + 1, train_loss) + time_str)
        print('Epoch: {}, Train Loss: {:.4f}, Valid Loss: {:.4f} '.format(e + 1, train_loss, valid_loss) + time_str)
    net.train()
    
final_loss = train_losses[-1]

ModuleNotFoundError: No module named 'AE'